# Import all the required dependancies

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf


In [ ]:
from tensorflow.keras.applications import EfficientNetB0


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zeyad1mashhour/driver-inattention-detection-dataset")

print("Path to dataset files:", path)

100%|██████████| 373M/373M [00:02<00:00, 148MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/zeyad1mashhour/driver-inattention-detection-dataset/versions/1


In [ ]:

# target_size = (224, 224)
# num_classes = 6

# train_folder = '/kaggle/input/driver-inattention-detection-dataset/train'
# test_folder = '/kaggle/input/driver-inattention-detection-dataset/test'
# valid_folder = '/kaggle/input/driver-inattention-detection-dataset/valid'

dataset_path = "/root/.cache/kagglehub/datasets/zeyad1mashhour/driver-inattention-detection-dataset/versions/1"

train_folder = os.path.join(dataset_path, "train")
valid_folder = os.path.join(dataset_path, "valid")
test_folder  = os.path.join(dataset_path, "test")

target_size = (224, 224)
num_classes = 6



# Data processing


In [ ]:
def read_annotations(file_path):
    annotations = []
    with open(file_path, "r") as file:
        for line in file:
            parts = line.strip().split()

            if len(parts) < 2:
                continue

            image_name = parts[0]

            try:
                bbox = list(map(int, parts[1].split(",")))
                if len(bbox) != 5:
                    continue

                annotations.append([image_name] + bbox)

            except ValueError:
                continue

    return pd.DataFrame(
        annotations,
        columns=["image_name", "x_min", "y_min", "x_max", "y_max", "class_id"]
    )


def preprocess_image(image_path, target_size):
    image = cv2.imread(image_path)
    if image is None:
        return None

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = preprocess_input(image)

    return image


def one_hot_encode_labels(labels, num_classes):
    encoder = OneHotEncoder(sparse_output=False)
    labels = np.array(labels).reshape(-1, 1)
    one_hot = encoder.fit_transform(labels)
    return one_hot


def preprocess_dataset(annotations, folder_path, num_classes):
    images = []
    labels = []

    for _, row in annotations.iterrows():
        image_path = os.path.join(folder_path, row["image_name"])
        image = preprocess_image(image_path, target_size)

        if image is None:
            continue

        images.append(image)
        labels.append(row["class_id"])

    images = np.array(images)
    one_hot_labels = one_hot_encode_labels(labels, num_classes)

    return images, one_hot_labels


# --- FIXED PATHS WITH KAGGLEHUB DATASET ---
train_annotations = read_annotations(os.path.join(train_folder, "_annotations.txt"))
valid_annotations = read_annotations(os.path.join(valid_folder, "_annotations.txt"))
test_annotations  = read_annotations(os.path.join(test_folder, "_annotations.txt"))

# --- PREPROCESS DATASET ---
train_images, train_labels = preprocess_dataset(train_annotations, train_folder, num_classes)
valid_images, valid_labels = preprocess_dataset(valid_annotations, valid_folder, num_classes)
test_images,  test_labels  = preprocess_dataset(test_annotations,  test_folder,  num_classes)


In [ ]:
print("Train images shape:", train_images.shape)
print("Train labels shape:", train_labels.shape)

print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

print("Valid images shape:", valid_images.shape)
print("Valid labels shape:", valid_labels.shape)


Train images shape: (11948, 224, 224, 3)
Train labels shape: (11948, 6)
Test images shape: (985, 224, 224, 3)
Test labels shape: (985, 6)
Valid images shape: (1922, 224, 224, 3)
Valid labels shape: (1922, 6)


# Assigning the data to Train and Validation

In [ ]:

(X_train, y_train), (X_val, y_val) = (train_images, train_labels), (valid_images, valid_labels)

# EfficientNetB0 model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model = tf.keras.models.load_model("/content/drive/MyDrive/stage1_model.keras")
print("Model loaded successfully!")

Mounted at /content/drive
Model loaded successfully!


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,038,007 (19.22 MB)

 Trainable params: 329,478 (1.26 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 658,958 (2.51 MB)

# Fine Tune stage 2 with 40 layers unfrozen

In [ ]:
# Fine-tuning
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True
class_weights = compute_class_weight('balanced',
                                    classes=np.unique(train_annotations['class_id']),
                                    y=train_annotations['class_id'])
class_weight_dict = dict(enumerate(class_weights))
for key in class_weight_dict:
    if class_weight_dict[key] < 1.0:
        class_weight_dict[key] *= 1.5

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Warmup GPU
dummy_data = tf.random.uniform((1, 224, 224, 3))
model.predict(dummy_data)


model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

for layer in base_model.layers[:-40]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(datagen.flow(X_train, y_train, batch_size=16),
                         epochs=8,
                         validation_data=(X_val, y_val),
                         callbacks=[reduce_lr, early_stopping],
                         class_weight=class_weight_dict)


1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
747/747 ━━━━━━━━━━━━━━━━━━━━ 215s 247ms/step - accuracy: 0.8488 - loss: 0.4387 - val_accuracy: 0.9116 - val_loss: 0.2965 - learning_rate: 2.0000e-05
Epoch 2/8
747/747 ━━━━━━━━━━━━━━━━━━━━ 146s 195ms/step - accuracy: 0.8635 - loss: 0.4216 - val_accuracy: 0.9105 - val_loss: 0.2983 - learning_rate: 2.0000e-05
Epoch 3/8
747/747 ━━━━━━━━━━━━━━━━━━━━ 151s 202ms/step - accuracy: 0.8574 - loss: 0.4367 - val_accuracy: 0.9131 - val_loss: 0.2923 - learning_rate: 2.0000e-05
Epoch 4/8
747/747 ━━━━━━━━━━━━━━━━━━━━ 147s 188ms/step - accuracy: 0.8659 - loss: 0.4104 - val_accuracy: 0.9121 - val_loss: 0.2926 - learning_rate: 2.0000e-05
Epoch 5/8
747/747 ━━━━━━━━━━━━━━━━━━━━ 141s 188ms/step - accuracy: 0.8647 - loss: 0.3990 - val_accuracy: 0.9147 - val_loss: 0.2929 - learning_rate: 2.0000e-05
Epoch 6/8
747/747 ━━━━━━━━━━━━━━━━━━━━ 141s 188ms/step - accuracy: 0.8714 - loss: 0.3870 - val_accuracy: 0.9095 - val_loss: 0.2924 - learning_rate: 2.0000e-05
Epoch 7/8
747/747 ━━━━━━━━━━━━━━━━━━━━ 143s 19

In [ ]:
model.save("/content/drive/MyDrive/stage2_model.keras")


# Model evaluation

In [ ]:

test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy:.4f}, Test loss: {test_loss:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 474ms/step - accuracy: 0.8974 - loss: 0.3058
Test accuracy: 0.9036, Test loss: 0.2952


In [ ]:
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test_labels, axis=1)


31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 278ms/step


In [ ]:
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)


In [ ]:
acc = accuracy_score(y_true_classes, y_pred_classes)
prec = precision_score(y_true_classes, y_pred_classes, average='macro')
rec = recall_score(y_true_classes, y_pred_classes, average='macro')
f1 = f1_score(y_true_classes, y_pred_classes, average='macro')
print(f"Accuracy: {acc:.4f}")
print(f"Precision (macro): {prec:.4f}")
print(f"Recall (macro): {rec:.4f}")
print(f"F1-score (macro): {f1:.4f}")


Accuracy: 0.9036
Precision (macro): 0.8615
Recall (macro): 0.8958
F1-score (macro): 0.8772
